In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import csv

# Run data setup script  
 - Processes original event data
 - Writes out new csv event data used for the Apache Cassandra tables

In [2]:
# Runs preprocessing setup script 
!python CassandraPreprocessing.py

Read in 8056 rows and wrote out 6821 rows to new event file


In [3]:
# new event data file
event_file = 'event_datafile_new.csv'

## Load event data into Cassanda database
### The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

## Setup Cluster and Keyspace 

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [5]:
session.execute("DROP KEYSPACE IF EXISTS parkify")
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
    print ('\n %s KEYSPACE created' % 'sparkify')
except Exception as e:
    print(e)


 sparkify KEYSPACE created


In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

# The create necessary Apache Cassanda tables and queries to answer the following three questions of the data:  

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## Question 1.

## Create table `song_length` to answer questions about specific session IDs and items itemInSession
 - The `song_length` table is modeled for responding the question #1 above which requires both session ID and the session item number.  
 - PK (session_id and item_in_session) - The session ID alone is not suffiecient for uniqueness, so a composite with item was used. 

In [7]:
# Uncomment below if need to replace table
# session.execute("DROP TABLE IF EXISTS song_length")

query = "CREATE TABLE IF NOT EXISTS song_length "
query = query + "(session_id int, item_in_session int, artist text, song text, length decimal,\
PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [8]:
# read in data from csv
with open(event_file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # parse rows and pull only needed columns 
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Question 1: <br>Select artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [9]:
query = "select artist, song, length from song_length where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print ("\nArtists, songs, and song length from session 338 with itemInSession = 4:\n")    
for row in rows:
    print ('\t', row.artist, row.song, row.length)


Artists, songs, and song length from session 338 with itemInSession = 4:

	 Faithless Music Matters (Mark Knight Dub) 495.3073



## Question 2.

## Create table `song_session` to answer questions about specific session IDs and items itemInSession
 - The `song_session` table is modeled for responding the question #2 above.  
 - PK (user_id and session_id) cluster key (item_in_session) - The composite of user and session IDs guarantee uniqueness and the item in the cluster allows for efficient sorting and data retrieval. 

In [10]:
# Uncomment below if need to replace table
# session.execute("DROP TABLE IF EXISTS song_session")

query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, user_first_name text, user_last_name text,\
PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [11]:
with open(event_file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_session (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # parse rows and pull only needed columns
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Question 2: <br>Select name of artist, song and user (first and last name) for userid = 10, sessionid = 182 <br>sorted by itemInSession    

In [12]:
query = "select artist, song, user_first_name, user_last_name \
from song_session where user_id = 10 and session_id = 182 \
ORDER by item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print ("Artists, songs, and users from session 182 and with userID 10 orderd by itemInSession:\n")    
for row in rows:
    print ('\t', row.artist, row.song)#, row.user_first_name, row.user_last_name)

Artists, songs, and users from session 182 and with userID 10 orderd by itemInSession:

	 Down To The Bone Keep On Keepin' On
	 Three Drives Greece 2000
	 Sebastien Tellier Kilometer
	 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)



## Question 3.

## Create table  `song_table` to answer questions about which users have listened to specific songs.
 - The `song_table` is modeled on question #3 above. 
 - PK (song and user_id) - The composite of song and userID guarantee  uniqueness and allow for efficient data retrieval, but only song name (partition key) is needed for filtering. 


In [13]:
# Uncomment below if need to replace table
#session.execute("DROP TABLE IF EXISTS song_users")

# song by user info (name, gender and level)
query = "CREATE TABLE IF NOT EXISTS song_users "
query = query + "(song text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY (song, user_id))" 
try:
    session.execute(query)
except Exception as e:
    print(e)


In [14]:
with open(event_file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # QA: uncomment below to validate the appropriate data is going into database - Not recomended for large datasets
        #if line[9] == 'All Hands Against His Own': print (line)
        query = "INSERT INTO song_users (song, user_id, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # parse rows and pull only needed columns
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Question 3: <br>Select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
query = "select user_first_name, user_last_name from song_users where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print ("Users who have listened to the song 'All Hands Against His Own':\n")
for row in rows:
    print ('\t', row.user_first_name, row.user_last_name)

Users who have listened to the song 'All Hands Against His Own':

	 Jacqueline Lynch
	 Tegan Levine
	 Sara Johnson


## Clean up 
### Drop the tables before closing out the sessions

In [16]:
# drop tables created above for answering the specific questions
for table in ['song_length', 'song_session', 'song_users']:
    query = "drop table %s" % table
    try:
        session.execute(query)
        print ('Dropped table %s' % table)
    except Exception as e:
        print(e)

Dropped table song_length
Dropped table song_session
Dropped table song_users


### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()

In [18]:
print ('Done, database clean and conections shutdown.')

Done, database clean and conections shutdown.
